In [2]:
import numpy as np

# transpose
a = np.array([[0, 1], [2, 3]])
a.transpose(1, 0)

array([[0, 2],
       [1, 3]])

In [8]:
a = np.array([[[0, 1], [2, 3]], [[4, 5], [6, 7]]])
a.transpose(0, 2, 1)

array([[[0, 2],
        [1, 3]],

       [[4, 6],
        [5, 7]]])

In [9]:
a.transpose()

array([[[0, 4],
        [2, 6]],

       [[1, 5],
        [3, 7]]])

In [7]:
np.pad([[1, 2], [3, 4]], [(0, 0), (1, 1)], mode="constant")

array([[0, 1, 2, 0],
       [0, 3, 4, 0]])

In [9]:
array = np.array([[1, 2, 3], [4, 5, 6]])
array[:, 0] = 10000
array

array([[10000,     2,     3],
       [10000,     5,     6]])

In [21]:
array = np.array(
    [
        [[[1, 2, 3, 4], [1, 2, 3, 4]], [[1, 2, 3, 4], [1, 2, 3, 4]]],
        [[[1, 2, 3, 4], [1, 2, 3, 4]], [[1, 2, 3, 4], [1, 2, 3, 4]]],
    ],
)

array.reshape(2, -1).T

array([[1, 1],
       [2, 2],
       [3, 3],
       [4, 4],
       [1, 1],
       [2, 2],
       [3, 3],
       [4, 4],
       [1, 1],
       [2, 2],
       [3, 3],
       [4, 4],
       [1, 1],
       [2, 2],
       [3, 3],
       [4, 4]])

In [ ]:
from common.util import col2im, im2col


class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad

        # 中間データ（backward時に使用）
        self.x = None
        self.col = None
        self.col_W = None

        # 重み・バイアスパラメータの勾配
        self.dW = None
        self.db = None

    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = 1 + int((H + 2 * self.pad - FH) / self.stride)
        out_w = 1 + int((W + 2 * self.pad - FW) / self.stride)

        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T

        out = np.dot(col, col_W) + self.b
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

        self.x = x
        self.col = col
        self.col_W = col_W

        return out

    def backward(self, dout):
        FN, C, FH, FW = self.W.shape
        dout = dout.transpose(0, 2, 3, 1).reshape(-1, FN)

        self.db = np.sum(dout, axis=0)
        self.dW = np.dot(self.col.T, dout)
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)

        dcol = np.dot(dout, self.col_W.T)
        dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)

        return dx


class Pooling:
    def __init__(self, pool_h, pool_w, stride=2, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad

        self.x = None
        self.arg_max = None

    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)

        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h * self.pool_w)

        arg_max = np.argmax(col, axis=1)
        out = np.max(col, axis=1)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        self.x = x
        self.arg_max = arg_max

        return out

    def backward(self, dout):
        dout = dout.transpose(0, 2, 3, 1)

        pool_size = self.pool_h * self.pool_w
        dmax = np.zeros((dout.size, pool_size))
        dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
        dmax = dmax.reshape(dout.shape + (pool_size,))

        dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
        dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)

        return dx

In [2]:
# coding: utf-8
import sys, os

sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from common.simple_convnet import SimpleConvNet
from common.trainer import Trainer

# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

# 処理に時間のかかる場合はデータを削減
# x_train, t_train = x_train[:5000], t_train[:5000]
# x_test, t_test = x_test[:1000], t_test[:1000]

max_epochs = 20

network = SimpleConvNet(
    input_dim=(1, 28, 28),
    conv_param={"filter_num": 30, "filter_size": 5, "pad": 0, "stride": 1},
    hidden_size=100,
    output_size=10,
    weight_init_std=0.01,
)

trainer = Trainer(
    network,
    x_train,
    t_train,
    x_test,
    t_test,
    epochs=max_epochs,
    mini_batch_size=100,
    optimizer="Adam",
    optimizer_param={"lr": 0.001},
    evaluate_sample_num_per_epoch=1000,
)
trainer.train()

# パラメータの保存
network.save_params("params.pkl")
print("Saved Network Parameters!")

# グラフの描画
markers = {"train": "o", "test": "s"}
x = np.arange(max_epochs)
plt.plot(x, trainer.train_acc_list, marker="o", label="train", markevery=2)
plt.plot(x, trainer.test_acc_list, marker="s", label="test", markevery=2)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc="lower right")
plt.show()

train loss:2.2994083518085473
=== epoch:1, train acc:0.267, test acc:0.328 ===
train loss:2.295147521171212
train loss:2.292202156999552
train loss:2.285110752027592
train loss:2.2731138400256237
train loss:2.265909348062124
train loss:2.24680021052272
train loss:2.240895101797497
train loss:2.1854319182020783
train loss:2.1686192504713624
train loss:2.1491077014731994
train loss:2.1124391061222907
train loss:2.032659414206755
train loss:2.0325774384576056
train loss:1.91592261241753
train loss:1.8582380256932622
train loss:1.7801594688070292
train loss:1.6771385380136372
train loss:1.694398319814699
train loss:1.5231872624987495
train loss:1.5025233428163918
train loss:1.4249942869992611
train loss:1.2867288952986835
train loss:1.2411186562785226
train loss:1.2567596073394998
train loss:1.2180474800503416
train loss:1.0259517230425539
train loss:0.8462640238289068
train loss:0.9176088144555
train loss:0.9087201744252912
train loss:0.7513773754737417
train loss:0.6964857716963266
train

KeyboardInterrupt: 

TypeError: unsupported operand type(s) for +: 'int' and 'tuple'